[ELMo for Text Classification](https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/)

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

### Read and Inspect the Data

In [3]:
# read data
train = pd.read_csv("train_2kmZucJ.csv")
test = pd.read_csv("test_oJQbWVk.csv")

train.shape, test.shape

((7920, 3), (1953, 2))

In [4]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [5]:
train['label'].value_counts(normalize=True)

0    0.744192
1    0.255808
Name: label, dtype: float64

### Text Cleaning and Preprocessing

In [6]:
# remove URL's from train and test
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

In [7]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

# remove numbers
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

# remove whitespaces
train['clean_tweet'] = train['clean_tweet'].apply(lambda x:' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

In [8]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [9]:
train['clean_tweet'] = lemmatization(train['clean_tweet'])
test['clean_tweet'] = lemmatization(test['clean_tweet'])

In [10]:
train.sample(10)

,id,label,tweet,clean_tweet
5362,5363,0,Love my shoes #Nike #neon #blue #popular #run4fun #iPhone #good #instalove #instagood #instagram http://instagr.am/p/KoFLlmIT2r/,love -PRON- shoe nike neon blue popular run fun iphone good instalove instagood instagram
7532,7533,0,@Soffici #japan #beautiful #likes #iphone #ipad #cute #wow #flower #insta ... ?item=19314103,soffici japan beautiful likes iphone ipad cute wow flower insta ... item
2745,2746,0,Wooohooo!!! NOTE3!!! Yeah bitches!! #GALAXYNOTE3 #GALAXY #NOTE #3 #SAMSUNG #ANDROID #CELCOM #PHONE #EXCITED,wooohooo note yeah bitches galaxynote galaxy note samsung android celcom phone excite
4278,4279,1,"Thanks #apple for screwing up my #iPhone with your crampy, crappy #ios11 you and your #iOS !","thanks apple for screw up -PRON- iphone with -PRON- crampy , crappy ios -PRON- and -PRON- ios"
5394,5395,0,"Bye Iphone 4 , Hello Iphone 5C #apple #iphone5c #blue http://instagram.com/p/fk6ZlEjsqa/","bye iphone , hello iphone c apple iphone c blue"
5242,5243,0,"U luv sum1-U marry sum1 else-The 1 u marry becomes ur spouse&the 1 u loved becomes the password of ur mail id"" #women #ipad #iphone",u luv sum u marry sum elsethe u marry become ur spousethe u love become the password of ur mail -PRON- would woman ipad iphone
1666,1667,0,Does anyone know where I can make my phone look like this? #pink #iphone pic.twitter.com/AKOvoCIFSq,do anyone know where i can make -PRON- phone look like this pink iphone pic.twitter.comakovocifsq
4230,4231,0,Sounds of Thunderstorms for #iPhone and #iPad - #Meditation #Health #Relaxation #Life #Nature #Yoga https://goo.gl/zXTXV1 pic.twitter.com/HCQrKE55Ol,sound of thunderstorm for iphone and ipad meditation health relaxation life nature yoga pic.twitter.comhcqrke old
4017,4018,0,Good Night #me #myself #lastyear #cute #hipster #fun #mirror #samsung #pataturri #movilphone… http://instagram.com/p/fyHiOgpBJ3/,good night -PRON- -PRON- lastyear cute hipster fun mirror samsung pataturri movilphone …
2768,2769,0,Frank’s Kauai Tropical Paradise Come join me! http://frankskauaitp.blogspot.com/ #hawaii #Kauai #Vacation #Travel #Photography #iPhone #sugar Cane,frank ’s kauai tropical paradise come join -PRON- hawaii kauai vacation travel photography iphone sugar cane


# Brief Intro to TensorFlow Hub

In [1]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [14]:
def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [11]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [15]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [16]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [17]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [18]:

# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

# Model Building and Evaluation

In [19]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
preds_valid = lreg.predict(xvalid)

In [22]:
f1_score(yvalid, preds_valid)

0.7714285714285715